<a href="https://colab.research.google.com/github/franciscolpjr/Gemini/blob/main/My_first_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install -q -U google-generativeai

In [1]:
import numpy as np
import pandas as pd

import google.generativeai as genai
from google.colab import userdata

from google.colab import userdata
api_key = userdata.get('secret_api_key')
GOOGLE_API_KEY=api_key
genai.configure(api_key=GOOGLE_API_KEY)

In [25]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    #print(m.name)
    pass

model_name="models/embedding-001"

In [5]:
title="Lista de produtos"
sample_text = {
    "Item 1:" "['descrição':'cesta de palha tamanho M','valor':'R$ 25,00','fabricante':'Sta Catarina']," "\n"
    "Item 2:" "['descrição':'cesta de palha tamanho G','valor':'R$ 35,00','fabricante':'Sta Catarina']," "\n"
    "Item 3:" "['descrição':'vinho tinto malbec','valor':'R$ 45,00','fabricante':'Concha e Toro']," "\n"
    "Item 4:" "['descrição':'queijo minas artesanal','valor':'R$ 50,00','fabricante':'Celeiro do leite']," "\n"
    "Item 5:" "['descrição':'torradas de pão frances','valor':'R$ 15,00','fabricante':'Bauducco']," "\n"
}

embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")

#print(embeddings)

In [6]:
documento1 = {
    "Título":"Cesta de café da manhã romântico.",
    "Conteúdo":"Cesta de café da manhã preparado com todo cuidado para pessoa amada. Inclui itens como quitandas frescas, suco natural e um arranjo de flores naturais. Valor: R$190,00"
}

documento2 = {
    "Título":"Cesta de café da manhã para pessoas alegres.",
    "Conteúdo":"Cesta de café da manhã alegre. Contém vários itens para alegrar o dia todo desta pessoa, tais como: Café cappucino e pão de queijo. Duas cervejas long-neck com petiscos. E um cartão de bom dia. Valor: R$145,00"
}

documento3 = {
    "Título":"Cesta de happy hour.",
    "Conteúdo":"Cesta de happy hour para alegrar o final de dia de qualquer pessoa. Inclui itens como: 4 cervejas long-neck, 3 petiscos diversos e 2 pedaços de queijo. Valor: R$155,00"
}

documents = [documento1, documento2, documento3]

In [15]:
lista = pd.DataFrame(documents)
lista.columns = ["Titulo", "Conteudo"]
#print(lista)

                                         Titulo  \
0             Cesta de café da manhã romântico.   
1  Cesta de café da manhã para pessoas alegres.   
2                          Cesta de happy hour.   

                                            Conteudo  
0  Cesta de café da manhã preparado com todo cuid...  
1  Cesta de café da manhã alegre. Contém vários i...  
2  Cesta de happy hour para alegrar o final de di...  


In [20]:
def embeddingGen(title,text):
  return genai.embed_content(model=model_name,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

lista["embeddings"] = lista.apply(lambda row: embeddingGen(row["Titulo"],row["Conteudo"]),axis=1)

#print(lista)

                                         Titulo  \
0             Cesta de café da manhã romântico.   
1  Cesta de café da manhã para pessoas alegres.   
2                          Cesta de happy hour.   

                                            Conteudo  \
0  Cesta de café da manhã preparado com todo cuid...   
1  Cesta de café da manhã alegre. Contém vários i...   
2  Cesta de happy hour para alegrar o final de di...   

                                          embeddings  
0  [0.035240542, -0.05373278, -0.008759275, -0.02...  
1  [0.0055454, -0.021626735, -0.039256062, -0.014...  
2  [0.033032097, -0.0051309713, -0.028721677, -0....  


In [27]:
def consulta_fn(consulta,lista,model_name):
   embedding_da_consulta = genai.embed_content(model=model_name,content=consulta,task_type="RETRIEVAL_DOCUMENT")["embedding"]
   produtos_escalares = np.dot(np.stack(lista["embeddings"]),embedding_da_consulta)
   indice = np.argmax(produtos_escalares)
   return lista.iloc[indice]["Conteudo"]

In [33]:
consulta = "Quero uma cesta para minha amada esposa. Qual você indica?"

resultado = consulta_fn(consulta,lista,model_name)

print(resultado)

Cesta de café da manhã preparado com todo cuidado para pessoa amada. Inclui itens como quitandas frescas, suco natural e um arranjo de flores naturais. Valor: R$190,00


In [40]:
generation_config = {
    "temperature": 0.2,
    "candidate_count": 1,
    "max_output_tokens": 1000,
}

prompt = f"Aja como um bom vendedor e reescreva o conteúdo do texto sem adicionar informações. Texto:{resultado}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",generation_config=generation_config)
resultado_2 = model_2.generate_content(prompt)

print(resultado_2.text)

**Cesta de Café da Manhã Encantadora**

Imagine o sorriso no rosto da pessoa amada ao receber esta cesta de café da manhã cuidadosamente preparada.

Cada item foi selecionado com carinho para criar uma experiência inesquecível:

* Quitandas fresquinhas, assadas com amor
* Suco natural, repleto de vitaminas e sabor
* Arranjo de flores naturais, trazendo beleza e fragrância

Tudo isso por um valor especial de R$ 190,00.

Presenteie quem você ama com um café da manhã inesquecível, repleto de carinho e atenção.
